#### Split Test Analysis

Working on Cobra Kai, a fictional combat card game, you recently introduced a new special bundle containing various characters as an A/B/C/D pricing test.

Given the test details below, and the data included in the tab 'SECTION 2 DATA', please analyze the results of the test and provide recommendations on further actions.	
	
| Group &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  | Split&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; | Price&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; | Discount Ratio |
|:---    |-------|-------|----------------|
| Control | 10%   | N/A   | N/A            |
| Case A  | 30%   | $2,99 | 50%            |
| Case B  | 30%   | $6,99 | 50%            |
| Case C  | 30%   | $9,99 | 50%            |

&nbsp;  
• The test ran for 30 days on iOS only, targeting all users, starting on the 15/03/2017  
• The data includes 14 days before the start of the test, from 01/03 to 14/03  
• The bundle composition was adapted to provide the same discount value to each user  
• The car quality, type and class split were kept the same across the different bundles  
• The bundles consisted of completely new cars, which were only available to buy through these offers  
• The normal *HC packs of the game are priced \$ 1.99 / \$ 4.99 / \$ 9.99 / \$ 19.99 / \$ 49.99 / \$ 99.99  

**HC=hard currency (currency purchased with real money)*


### Data Preparation

In [1]:
# Importing libraries
!pip install openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [12]:
# Importing the data
df = pd.read_excel('./files/data/data_raw.xlsx', skiprows=4)
df.head()

,Date,Group,DAU,Revenue,PAU/DAU,PAU (Paying Active Users),ARPU,ARPPU,Transactions,Car bundle A // 2.99$,...,Pack 5 // 49.99$,Pack 6 // 99.99$,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,2017-03-01,control,343970,19794,0.007492,2577,0.057546,7.681024,5238.402004,0,...,3391.0,5868.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-03-02,control,340743,16067,0.007118,2426,0.047153,6.622836,4416.350040,0,...,2704.0,4861.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-03-03,control,399292,16212,0.007807,3117,0.040602,5.201155,5564.455360,0,...,2827.0,4917.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-03-04,control,413060,16824,0.007345,3034,0.040730,5.545155,5485.192860,0,...,2792.0,4995.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-03-05,control,388422,18257,0.006887,2675,0.047003,6.825047,4846.145624,0,...,3188.0,5552.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Date                       176 non-null    datetime64[ns]
 1   Group                      176 non-null    object        
 2   DAU                        176 non-null    int64         
 3   Revenue                    176 non-null    int64         
 4   PAU/DAU                    176 non-null    float64       
 5   PAU (Paying Active Users)  176 non-null    int64         
 6   ARPU                       176 non-null    float64       
 7   ARPPU                      176 non-null    float64       
 8   Transactions               176 non-null    float64       
 9   Car bundle A // 2.99$      176 non-null    int64         
 10  Car bundle B // 6.99$      176 non-null    int64         
 11  Car bundle C // 9.99$      176 non-null    int64         
 12  Pack 1 /

In [14]:
# Dropping unnecesary columns
df = df.iloc[:,:18]

In [15]:
# Renaming PAU column
df.rename(columns={'PAU (Paying Active Users)': 'PAU'}, inplace = True)

In [16]:
# Setting a new column with all the promotions sold
df['Card bundle promotion'] = df['Car bundle A // 2.99$'] + df['Car bundle B // 6.99$'] + df['Car bundle C // 9.99$']

# Setting a new column with number of promotion packs sold
df['Promotions_sold'] = df['Car bundle A // 2.99$']/2.99 + df['Car bundle B // 6.99$']/6.99 + df['Car bundle C // 9.99$']/9.99

# Setting a new column with sales not in promotion
df['Sales not in promotion'] = df['Revenue'] - df['Card bundle promotion']

In [17]:
# Setting the date as index and formatting it
df.set_index('Date', inplace=True)
df.index = df.index.strftime('%m/%d/%Y')

In [19]:
# Defining column that indicates if the measures are before or during the experiment 
df['Timing'] = ["Previous" if x < '03/15/2017' else "Post" for x in df.index]

In [21]:
# Checking it's alright
df.head()

,Group,DAU,Revenue,PAU/DAU,PAU,ARPU,ARPPU,Transactions,Car bundle A // 2.99$,Car bundle B // 6.99$,...,Pack 1 // 1.99$,Pack 2 // 4.99$,Pack 3 // 9.99$,Pack 4 // 19.99$,Pack 5 // 49.99$,Pack 6 // 99.99$,Card bundle promotion,Promotions_sold,Sales not in promotion,Timing
Date,,,,,,,,,,,,,,,,,,,,,
03/01/2017,control,343970,19794,0.007492,2577,0.057546,7.681024,5238.402004,0,0,...,1484.0,1356.0,2745.0,4950.0,3391.0,5868.0,0,0.0,19794,Previous
03/02/2017,control,340743,16067,0.007118,2426,0.047153,6.622836,4416.350040,0,0,...,1051.0,1183.0,2290.0,3978.0,2704.0,4861.0,0,0.0,16067,Previous
03/03/2017,control,399292,16212,0.007807,3117,0.040602,5.201155,5564.455360,0,0,...,977.0,1205.0,2208.0,4078.0,2827.0,4917.0,0,0.0,16212,Previous
03/04/2017,control,413060,16824,0.007345,3034,0.040730,5.545155,5485.192860,0,0,...,1303.0,1235.0,2354.0,4145.0,2792.0,4995.0,0,0.0,16824,Previous
03/05/2017,control,388422,18257,0.006887,2675,0.047003,6.825047,4846.145624,0,0,...,1065.0,1298.0,2535.0,4619.0,3188.0,5552.0,0,0.0,18257,Previous


In [23]:
# Exporting processed data into a csv file
df.to_csv('./files/data/data_processed.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c0f6dcf0-1a26-4fa8-bfda-9393dc33e48a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>